# To do list:
- first see how you can specify the specific gpu to be used by ollama, maybe their doc helps
- see why you cannot get good results with llama2 and llama3.1, how can i make the rag app better to make answer and not empty str


different models supportyed in Ollama: https://github.com/ollama/ollama 

# Libraries Required for the Excercise

In [3]:
%%capture --no-stderr
! pip install langsmith langchain-community langchain chromadb tiktoken langchain_openai chainlit
!npm install localtunnel 

## Setup Required - APIs

In this notebook you'll be using OpenAI models for reasoning and generation, langsmith for tracing and langchain for orchestration, chainlit for UI.

In [2]:
import os
from langsmith import utils
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

import openai
from langsmith import traceable

import getpass

from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# define your langchain project name here
os.environ["LANGCHAIN_PROJECT"] = "maven-course-learning-path"

print('Is Tracing is enabled:', utils.tracing_is_enabled())

Is Tracing is enabled: True


In [1]:
!nvidia-smi


Tue Dec 17 00:04:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:2B:00.0 Off |                   On |
| N/A   53C    P0    81W / 300W |   2353MiB / 81920MiB |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  Off  | 00000000:2F:00.0 Off |                   On |
| N/A   

In [4]:

os.environ["CUDA_VISIBLE_DEVICES"] 


'0'

In [ ]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"Enter your {key}:")


_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

## Data Processing

In [2]:
try:
    from langchain_community.embeddings import OllamaEmbeddings
    print("Ollama is installed")
except ImportError:
    print("Ollama is not installed")


Ollama is installed


In [3]:
!ollama --version


ollama version is 0.5.2


In [5]:
from langchain_community.embeddings import OllamaEmbeddings
embedding = OllamaEmbeddings()
query = "Retrieval augmented generation is a technique used in NLP."
embedding_result = embedding.embed_query(query)
print(embedding_result)


[-0.07322676479816437, -0.07432803511619568, 0.09370782971382141, 0.22812822461128235, 0.16550856828689575, 0.017536558210849762, -0.07045720517635345, 0.09860721975564957, -0.18877196311950684, 0.23341260850429535, 0.23763073980808258, -0.1782284677028656, 0.18477687239646912, 0.06797169148921967, -0.009372031316161156, -0.034885600209236145, 0.10378145426511765, 0.1518734097480774, 0.22010278701782227, -0.13698913156986237, 0.012856383807957172, 0.14249370992183685, -0.05837327241897583, -0.29132047295570374, 0.02330014854669571, 0.04830487444996834, 0.21811717748641968, 0.17776930332183838, -0.127202570438385, 0.03757036104798317, 0.11763126403093338, -0.047713104635477066, 0.04899125173687935, -0.09663340449333191, -0.032046154141426086, -0.0011016528587788343, 0.058759383857250214, 0.000380201410735026, 0.026433050632476807, 0.15979509055614471, -0.023475632071495056, 0.10065437853336334, 0.1470029205083847, 0.15377673506736755, 0.3021175265312195, 0.08561335504055023, -0.28719508

In [5]:
from langchain_community.embeddings import OllamaEmbeddings 
embedding = OllamaEmbeddings()
embedding = OllamaEmbeddings(model="llama3.1")  # Specify the model you want, e.g., "llama2"

query = "Retrieval augmented generation is a technique used in NLP."
embedding_result = embedding.embed_query(query)
print(embedding_result)

/tmp/ipykernel_194325/2496812163.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings()


[-2.7002623081207275, -1.073585033416748, 1.5424476861953735, 0.4474245011806488, -2.2702879905700684, -2.2837836742401123, 1.4284785985946655, 1.264024257659912, -0.7373412251472473, -2.017709493637085, 1.5830085277557373, 0.010174687951803207, 0.3570461571216583, 0.726728618144989, 0.7599831223487854, 1.4779396057128906, -3.005201578140259, 0.7033123970031738, -0.6730121374130249, -1.236040711402893, -0.704370379447937, -2.344184160232544, -3.5333611965179443, 1.8580492734909058, 1.0354713201522827, 0.6671758890151978, 2.324751853942871, -0.6376690864562988, -0.5054599046707153, -1.5130754709243774, -0.30536454916000366, 3.2726893424987793, -0.8337347507476807, 3.240793466567993, 3.3887810707092285, -0.856768012046814, 0.362095445394516, 1.762982726097107, 2.9084105491638184, 0.4189751148223877, -0.16950881481170654, -0.506339967250824, -0.2194717526435852, 3.8209900856018066, -0.8299651145935059, -1.3539702892303467, 1.3238716125488281, 1.5637843608856201, 1.0441614389419556, -1.708

In [3]:

from bs4 import BeautifulSoup as Soup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.embeddings import OllamaEmbeddings

# Load
url = "https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# Specify the model you want to use, e.g., BERT
# embedding = HuggingFaceEmbeddings(model_name="bert-base-uncased")
# embeddings = OllamaEmbeddings()
embeddings = OllamaEmbeddings(model="llama3.1")  # Specify the model you want, e.g., "llama2"

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Index
retriever = vectorstore.as_retriever()

In [7]:
# Quick Check: Query the retriever with a sample question or text using the `invoke` method
query = "What is Retrieval-Augmented Generation (RAG)?"
result = retriever.invoke(query)  # Using `invoke` instead of `get_relevant_documents`

# Print the results to see if embeddings and retrieval are working
for doc in result:
    print(doc.page_content)


Comments - How to do retrieval augmented generation (RAG) right!
Comments - How to do retrieval augmented generation (RAG) right!
How to do retrieval augmented generation (RAG) right!
the Retrieve module fetches potentially relevant information from external data sources. This could involve keyword searching over a set of documents, extracting specific passages, querying structured databases via APIs, or even writing SQL-type queries. This could also involve a mixture of all of


In [ ]:
# this is what is used in the original file where they relied on OpenAI
### INDEX

from bs4 import BeautifulSoup as Soup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Load
url = "https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Index
retriever = vectorstore.as_retriever()

## RAG Application

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain import hub
class RagApp:

    def __init__(self, retriever, model: str = "llama3.1", prompt: str = "rlm/rag-prompt"):
        self._retriever = retriever
        self._llm = ChatOllama(model=model)
        self._prompt = hub.pull(prompt)

    @traceable()
    def retrieve_docs(self, question):
        return self._retriever.invoke(question)

    @traceable()
    def get_answer(self, question: str):
        rag_chain = (
            {"context": self._retriever, "question": RunnablePassthrough()}
            | self._prompt
            | self._llm
        )

        response = rag_chain.invoke(question)
        similar = self.retrieve_docs(question)
        return {
            "answer": response.content,
            "contexts": [str(doc) for doc in similar],
        }


# Example of usage
rag_bot = RagApp(retriever)


/tmp/ipykernel_188658/1264233552.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self._llm = ChatOllama(model=model)


In [ ]:
# this is what is used in the original file where they relied on OpenAI
# RAG
class RagApp:

    def __init__(self, retriever, model: str = "gpt-4o-mini", prompt: str = "rlm/rag-prompt"):
        self._retriever = retriever
        self._llm = ChatOpenAI(model = model)
        # Prompt Link: https://smith.langchain.com/hub/rlm/rag-prompt)
        self._prompt = hub.pull(prompt)

    @traceable()
    def retrieve_docs(self, question):
        return self._retriever.invoke(question)

    @traceable()
    def get_answer(self, question: str):
        rag_chain = (
            {"context": self._retriever, "question": RunnablePassthrough()}
            | self._prompt
            | self._llm
        )

        response = rag_chain.invoke(question)
        similar = self.retrieve_docs(question)
        #Evaluators will expect "answer" and "contexts"
        return {
            "answer": response.content,
            "contexts": [str(doc) for doc in similar],
        }

rag_bot = RagApp(retriever)

In [5]:
rag_response = rag_bot.get_answer("Name all individuals whose presentation insipred this blog")

In [6]:
rag_response

{'answer': '',
 'contexts': ['page_content=\'the Retrieve module fetches potentially relevant information from external data sources. This could involve keyword searching over a set of documents, extracting specific passages, querying structured databases via APIs, or even writing SQL-type queries. This could also involve a mixture of all of\' metadata={\'content_type\': \'text/html; charset=utf-8\', \'description\': \'My client: “hey, we have this vector db + LLM RAG thing and it’s not working”. And my answer is often “pull in the chair and sit down, we need to talk about how robust software is built".\', \'language\': \'en\', \'source\': \'https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation\', \'title\': \'How to do retrieval augmented generation (RAG) right!\'}',
  'page_content=\'for Large Language Models: A Survey - arXiv:2312.10997v5Common modulesOf course, in reality, the modules will end up largely replicating the steps a human would take to carry out a particu

## Create Front-end with Chainlit

In [11]:
%%writefile app.py
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
import chainlit as cl

from bs4 import BeautifulSoup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
import os

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_retriever():
    # Load
    url = "https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation"
    loader = RecursiveUrlLoader(
        url=url,
        max_depth=10,
        extractor=lambda x: BeautifulSoup(x, "html.parser").text
    )
    docs = loader.load()

    # Split
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)

    # Embed
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

    # Return retriever with search kwargs
    return vectorstore.as_retriever(search_kwargs={"k": 3})

@cl.on_chat_start
async def start():
    # Initialize the chain components
    retriever = get_retriever()
    llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
    prompt = hub.pull('rlm/rag-prompt')
    # Create the chain
    rag_chain = (
        {
            "context": lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )

    # Store the chain in the user session
    cl.user_session.set("chain", rag_chain)

    # Send an initial message
    await cl.Message(content="Hi! I'm ready to help you with questions about AISC substack article on RAG. What would you like to know?").send()

@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")  # Get the chain from the session

    if chain is None:
        await cl.Message(content="Error: Chain not initialized. Please restart the chat.").send()
        return

    msg = cl.Message(content="")

    async with cl.Step("Fetched Content 🤔"):
        async for chunk in chain.astream(
            {"question": message.content},
            config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
        ):
            await msg.stream_token(chunk.content)

    await msg.send()

if __name__ == "__main__":
    cl.run()

Writing app.py


### Access Chainlit service running on the colab instance using [localtunnel](https://github.com/localtunnel/localtunnel)

#### You'll need to enter the following password to access the dashboard

In [12]:
print('your password is: ')
!curl ipecho.net/plain

your password is: 
34.46.111.8

In [13]:
!chainlit run app.py & npx localtunnel --port 8000

2024-11-23 07:31:15 - Created default config file at /content/.chainlit/config.toml
2024-11-23 07:31:15 - Created default translation directory at /content/.chainlit/translations
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/ta.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/he-IL.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/gu.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/kn.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/hi.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/mr.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/ml.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/en-US.json
2024-11-23 07:31:15 - Created d